In [0]:
                                          ## Initialize
!pip install geopandas

import geopandas as gpd
import numpy as np
import pandas as pd
import time

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
## Load data
## https://data.colorado.gov/Demographics/Census-Blocks-in-Colorado-2010/xipb-k5bu
workdir='gdrive/My Drive/Project/GoCode 2019/Go Code - Regis Voter/Data/Raw datasets/'
df_cbg = gpd.read_file(workdir+'Census Block Groups in Colorado 2010.geojson')
print(df_cbg.tail())

     pct_amind      intptlon households med_age occupied v_s_notocc white_nh  \
3527       0.4  -104.8419816      779.0    49.7    779.0        0.0   1169.0   
3528      0.87  -104.7994864      613.0    34.0    613.0        0.0   1026.0   
3529      0.43  -104.7965144      582.0    34.4    582.0        0.0    913.0   
3530      0.59  -104.7962523      433.0    33.0    433.0        0.0    806.0   
3531       0.7  -104.6895702      617.0    30.4    617.0        3.0   1224.0   

     female state gq_o_noni  \
3527  938.0    08       0.0   
3528  907.0    08       0.0   
3529  822.0    08       0.0   
3530  659.0    08       0.0   
3531  922.0    08       0.0   

                            ...                         gq_nurs  owner avg_hh  \
3527                        ...                             0.0  593.0   2.25   
3528                        ...                             0.0  486.0    3.0   
3529                        ...                             0.0  446.0   2.77   
3530    

In [0]:
## https://data.colorado.gov/Transportation/Counties-in-Colorado/67vn-ijga
df_county =pd.read_csv(workdir+'COUNTIES.csv')
df_county_clean = df_county[['COUNTY','CO_FIPS']]

In [0]:
#Format new CensusBlock columns
df_cbg['Centroid']=df_cbg['geometry'].centroid
df_cbg['Lon']=df_cbg['geometry'].centroid.x
df_cbg['Lat']=df_cbg['geometry'].centroid.y
df_cbg2 = df_cbg[['county','tract','blkgrp','geometry','Centroid','Lat','Lon']]
df_cbg2['county'] = df_cbg2['county'].astype(int)
df_cbg_clean = df_cbg2.merge(df_county_clean, left_on = "county", right_on = 'CO_FIPS')
df_cbg_clean = df_cbg_clean.rename(index=str, columns={'COUNTY':'CountyName','tract':'Tract','geometry':'Geometry','blkgrp':'CensusBlockGroup'})
df_cbg_clean = df_cbg_clean[df_cbg_clean['CountyName']=='DENVER'] #CO_FIPS==31
df_cbg_clean = df_cbg_clean.drop(columns=['county','CO_FIPS','CountyName'])
df_cbg_clean['Tract-CensusBlockGroup'] = '108031'+df_cbg_clean['Tract'].astype(str)+df_cbg_clean['CensusBlockGroup'].astype(str)
df_cbg_clean.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Tract,CensusBlockGroup,Geometry,Centroid,Lat,Lon,Tract-CensusBlockGroup
1812,003601,4,(POLYGON ((-104.9634390000001 39.7657500000000...,POINT (-104.9659293039469 39.76695096702992),39.766951,-104.965929,1080310036014
1813,002702,4,(POLYGON ((-104.9867450000001 39.7320400000000...,POINT (-104.9855038771376 39.7296613713856),39.729661,-104.985504,1080310027024
1814,015600,3,(POLYGON ((-105.0250510000001 39.6749540000000...,POINT (-105.0203821298803 39.67522171545843),39.675222,-105.020382,1080310156003
1815,002701,4,(POLYGON ((-104.9823000000001 39.7351520000000...,POINT (-104.9835506253264 39.73604168173959),39.736042,-104.983551,1080310027014
1816,003601,5,(POLYGON ((-104.9684250000001 39.7657570000000...,POINT (-104.9708792307829 39.7669621567836),39.766962,-104.970879,1080310036015


In [0]:
# Saving the new merged file/checkpoint
def write_checkpoint(df, dfname):
  filename= dfname+time.strftime("%Y%m%d-%H%M")+'.csv'
  df.to_csv(filename, index=False)
  return

def read_checkpoint(filename):
    return pd.read_csv(filename,index_col=0)
  
write_checkpoint(df_cbg_clean,workdir+'DenverCensusBlockGroup_Geoinfo')